In [4]:
import tensorflow as tf 
from tensorflow.keras.preprocessing.text import Tokenizer

In [2]:
from google. colab import drive 

In [34]:
drive.mount ("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
x = ["I am very happy today", "No, I do not like the movie"]
y = [1,0]

In [26]:
#Create a tokenizer, configured to only take into account the 20 most common words
tokenizer = Tokenizer (num_words=20)

In [27]:
#Build the word index 
tokenizer.fit_on_texts(x)

In [28]:
#Turns strings into lists of integer indices
sequences = tokenizer. texts_to_sequences(x)

In [29]:
# show the vocabulory
word_index = tokenizer.word_index
print(tokenizer.word_index)
print('Found % unique tokens.' % len(word_index))

{'i': 1, 'am': 2, 'very': 3, 'happy': 4, 'today': 5, 'no': 6, 'do': 7, 'not': 8, 'like': 9, 'the': 10, 'movie': 11}
Found  11nique tokens.


In [30]:
print(sequences)

[[1, 2, 3, 4, 5], [6, 1, 7, 8, 9, 10, 11]]


In [31]:
# Identify max length of reviews
max_length = 0
for review_number in range(len(sequences)): 
  numberofwords=len(sequences[review_number])
  if (numberofwords) > (max_length):
    max_length = numberofwords
print(max_length)

7


In [33]:
# Padding the sequence for shorter reviews
from tensorflow.keras.preprocessing.sequence import pad_sequences 
import numpy as np
data = pad_sequences(sequences, maxlen=max_length)
y = np.asarray(y)
print(data)
print('Shape of data tensor : ',data.shape)
print('Shape of label tensor:',y.shape)


[[ 0  0  1  2  3  4  5]
 [ 6  1  7  8  9 10 11]]
Shape of data tensor :  (2, 7)
Shape of label tensor: (2,)


In [36]:
#Load my word embeddings
import gensim
wordembeddings = gensim.models.KeyedVectors.load_word2vec_format('drive/My Drive/GoogleNews-vectors-negative300.bin',binary=True)

In [41]:
#Converting data to matrix of shape (12,300) where 12 is number of words in vocaboulry + 1 and 300 word vector for each word
unique_words = len(word_index)
total_words = unique_words + 1
skipped_words = 0
embedding_dim = 300
embedding_matrix = np.zeros((total_words, embedding_dim))
for word, index in tokenizer.word_index.items():
  try:
    embedding_vector = wordembeddings[word]
  except:
    skipped_words = skipped_words+1
    pass
  if embedding_vector is not None:
    embedding_matrix[index] = embedding_vector
print("Embeddings Matrix shape : ",embedding_matrix.shape)


Embeddings Matrix shape :  (12, 300)


In [42]:
embedding_matrix[3]-wordembeddings.wv['very']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [43]:
from tensorflow.keras.layers import *
# create the embedding layer
embedding_Layer = Embedding(total_words, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False)

In [46]:
from tensorflow. keras.models import *
from tensorflow. keras. layers import *
# define model
model = Sequential()
model.add(embedding_Layer)
model.add(SimpleRNN(128, activation='relu', return_sequences= True)) 
model.add(SimpleRNN(256, activation='relu')) 
model.add(Dense(1, activation= 'sigmoid'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 7, 300)            3600      
                                                                 
 simple_rnn (SimpleRNN)      (None, 7, 128)            54912     
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 256)               98560     
                                                                 
 dense (Dense)               (None, 1)                 257       
                                                                 
Total params: 157,329
Trainable params: 153,729
Non-trainable params: 3,600
_________________________________________________________________
None


In [48]:
from keras.optimizers import *
# compile network
opt = SGD(lr=0.01, decay=1e-6)
model.compile(loss = "binary_crossentropy", optimizer = opt, metrics=['accuracy'])
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(data, y, epochs=10, verbose=1)

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 0.7007 - accuracy: 0.5000
Epoch 2/10
1/1 [==============================] - 0s 17ms/step - loss: 0.6714 - accuracy: 0.5000
Epoch 3/10
1/1 [==============================] - 0s 16ms/step - loss: 0.6448 - accuracy: 0.5000
Epoch 4/10
1/1 [==============================] - 0s 15ms/step - loss: 0.6220 - accuracy: 0.5000
Epoch 5/10
1/1 [==============================] - 0s 16ms/step - loss: 0.6015 - accuracy: 0.5000
Epoch 6/10
1/1 [==============================] - 0s 17ms/step - loss: 0.5822 - accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 20ms/step - loss: 0.5644 - accuracy: 1.0000
Epoch 8/10
1/1 [==============================] - 0s 16ms/step - loss: 0.5477 - accuracy: 1.0000
Epoch 9/10
1/1 [==============================] - 0s 15ms/step - loss: 0.5326 - accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - 0s 19ms/step - loss: 0.5175 - accuracy: 1.0000
